## How well can the level of corruption of a country in Europe be quantified? 

* What differences are there in actual corruption and perceived corruption? 

* Are there different forms of corruption prevalent in different countries in Europe? 

* What characteristics of a country predict the level of corruption? 

* What characteristics of a country predict an increase or decrease in the level of corruption?

• Happiness Report 2005-2022


In [45]:
import pandas as pd
import os

In [47]:
happiness_raw_data = pd.read_csv("../data/raw/happiness_report.csv")

In [49]:
happiness_raw_data.head()

,Country Name,Regional Indicator,Year,Life Ladder,Log GDP Per Capita,Social Support,Healthy Life Expectancy At Birth,Freedom To Make Life Choices,Generosity,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government
0,Afghanistan,South Asia,2008,3.723590,7.350416,0.450662,50.500000,0.718114,0.167652,0.881686,0.414297,0.258195,0.612072
1,Afghanistan,South Asia,2009,4.401778,7.508646,0.552308,50.799999,0.678896,0.190809,0.850035,0.481421,0.237092,0.611545
2,Afghanistan,South Asia,2010,4.758381,7.613900,0.539075,51.099998,0.600127,0.121316,0.706766,0.516907,0.275324,0.299357
3,Afghanistan,South Asia,2011,3.831719,7.581259,0.521104,51.400002,0.495901,0.163571,0.731109,0.479835,0.267175,0.307386
4,Afghanistan,South Asia,2012,3.782938,7.660506,0.520637,51.700001,0.530935,0.237588,0.775620,0.613513,0.267919,0.435440


In [51]:
countries = pd.read_csv("../data/processed/europe_countries.csv")

In [53]:
countries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Country    48 non-null     object
 1   ISO3 Code  48 non-null     object
 2   ISO2 Code  48 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [55]:
iso3_europe_all = set(countries["Country"])
len(iso3_europe_all)

48

In [57]:
iso3_europe_hap = set(happiness_raw_data["Country Name"])
len(iso3_europe_hap)

165

In [59]:
iso3_europe_all-iso3_europe_hap

{'Andorra',
 'Bosnia & Herzegovina',
 'Czech Republic',
 'Liechtenstein',
 'Monaco',
 'San Marino',
 'Turkey',
 'Vatican City'}

In [61]:
happiness_raw_data = happiness_raw_data[happiness_raw_data["Country Name"].isin(iso3_europe_all)]

In [63]:
happiness_raw_data.head()

,Country Name,Regional Indicator,Year,Life Ladder,Log GDP Per Capita,Social Support,Healthy Life Expectancy At Birth,Freedom To Make Life Choices,Generosity,Perceptions Of Corruption,Positive Affect,Negative Affect,Confidence In National Government
14,Albania,Central and Eastern Europe,2007,4.634252,9.121704,0.821372,66.760002,0.528605,-0.010429,0.874700,0.488819,0.246335,0.300681
15,Albania,Central and Eastern Europe,2009,5.485470,9.241429,0.833047,67.320000,0.525223,-0.159259,0.863665,0.564474,0.279257,NaN
16,Albania,Central and Eastern Europe,2010,5.268937,9.282793,0.733152,67.599998,0.568958,-0.173675,0.726262,0.576077,0.300060,NaN
17,Albania,Central and Eastern Europe,2011,5.867422,9.310619,0.759434,67.879997,0.487496,-0.206186,0.877003,0.565759,0.256577,NaN
18,Albania,Central and Eastern Europe,2012,5.510124,9.326344,0.784502,68.160004,0.601512,-0.170467,0.847675,0.553473,0.271393,0.364894


In [65]:
happiness_raw_data["Country Name"].nunique()

40

In [67]:
happiness_raw_data.to_csv("../data/processed/happiness.csv", index=False, index_label=False)

In [69]:
happiness = pd.read_csv("../data/processed/happiness.csv")

In [71]:
happiness.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 612 entries, 0 to 611
Data columns (total 13 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Country Name                       612 non-null    object 
 1   Regional Indicator                 612 non-null    object 
 2   Year                               612 non-null    int64  
 3   Life Ladder                        612 non-null    float64
 4   Log GDP Per Capita                 609 non-null    float64
 5   Social Support                     612 non-null    float64
 6   Healthy Life Expectancy At Birth   596 non-null    float64
 7   Freedom To Make Life Choices       611 non-null    float64
 8   Generosity                         594 non-null    float64
 9   Perceptions Of Corruption          606 non-null    float64
 10  Positive Affect                    611 non-null    float64
 11  Negative Affect                    612 non-null    float64